## 데이터 전처리

In [1]:
# base
import time
import pandas as pd
import re
import random
import numpy as np
from itertools import product

# visualization
from sklearn.manifold import TSNE
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

# word2vec
from gensim.models.word2vec import Word2Vec
import gensim 
import gensim.models as g
plt.rc('font', family='nanumgothic')
plt.rc('axes', unicode_minus=False)

# PCA
from sklearn.decomposition import PCA

# ignore
import warnings
warnings.filterwarnings("ignore")

# KMEANS
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [2]:
data = pd.read_csv('Youtube 이용 관련 온라인 실태조사(응답).csv', engine='python', encoding='cp949')
data.head()

,타임스탬프,1. 본인의 나이는 몇살인가요?,"2. 본인의 전공 혹은 직업은 무엇인가요? (예시 - 빅데이터경영통계전공, 경영학전공 / 간호사, 직장인)",3. 색조 화장품을 구매할 의향이 있으신가요?,4. 색조 화장품을 구매할 때 다양한 브랜드를 고려하시나요?,5. 유튜브를 시청 할 때 어떤 목적으로 보시나요?,하단 보기에서 본인이 관심있는 분야라고 생각되는 것을 모두 골라주세요.,유튜브에서 '건강' 관련된 영상을 시청하시나요?,1. '건강' 카테고리에서 보는 영상을 모두 체크해주세요,2. '건강' 관련 영상을 시청하는 정도를 체크해주세요,...,"4. '취미, 레저' 관련 영상에서 자주 시청하는 유튜버가 있으면 적어주세요",유튜브에서 '홈 및 가든' 관련된 영상을 시청하시나요?,1. '홈 및 가든' 보는 영상을 모두 체크해주세요,2. '홈 및 가든' 관련 영상을 시청하는 정도를 체크해주세요,3. '홈 및 가든' 관련 영상에서 주로 시청하는 콘텐츠를 모두 체크해주세요,4. '홈 및 가든' 영상에서 자주 시청하는 유튜버가 있으면 적어주세요,1. 화장품 브랜드 '클리오'를 알고계신가요?,2. '클리오' 제품을 구매해보신 적이 있나요?,3. '클리오' 제품을 앞으로도 계속 이용할 의향이 있으신가요?,4. 수고하셨습니다 :) 설문지에 응답해주셔서 감사드립니다. 경품 추첨을 위하여 번호를 적어주세요! 추첨하여 연락드릴 용도로만 쓰입니다. (경품은 정말 드립니다!)
0,5-19-2019 11:24:33,19,글로벌리더학과,네,네,정보 탐색을 위해서,"엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, 독서, 만화 및 애니메...",네,"건강 상태, 여성 건강, 정신 건강, 영양",3.0,...,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네,01028034347
1,5-19-2019 13:26:34,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,독서,아니오,NaN,NaN,...,NaN,아니오,NaN,NaN,NaN,NaN,아니오,아니오,아니오,설문 수준 실화입니까? 도대체가 이해가 안가네요 ^^ 이런것들이 여성들에게 꾸밈노동...
2,5-20-2019 12:01:43,19,치의예과,네,네,정보 탐색을 위해서,"독서, 영화, 음악, 여행",아니오,NaN,NaN,...,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네,01030866836
3,5-21-2019 13:11:03,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, TV, 독서, 만화 및...",아니오,NaN,NaN,...,"사나고, 이연",아니오,NaN,NaN,NaN,NaN,네,네,네,01035454833
4,5-22-2019 1:08:43,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","애완동물, 예술 및 공연, 패션, TV, 영화, 음악, 미용, 건강 및 피트니스, ...",아니오,NaN,NaN,...,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네,01091012066


In [3]:
data1 = data.copy()
data1 = data1.iloc[:,1:-1]
data1.head()

,1. 본인의 나이는 몇살인가요?,"2. 본인의 전공 혹은 직업은 무엇인가요? (예시 - 빅데이터경영통계전공, 경영학전공 / 간호사, 직장인)",3. 색조 화장품을 구매할 의향이 있으신가요?,4. 색조 화장품을 구매할 때 다양한 브랜드를 고려하시나요?,5. 유튜브를 시청 할 때 어떤 목적으로 보시나요?,하단 보기에서 본인이 관심있는 분야라고 생각되는 것을 모두 골라주세요.,유튜브에서 '건강' 관련된 영상을 시청하시나요?,1. '건강' 카테고리에서 보는 영상을 모두 체크해주세요,2. '건강' 관련 영상을 시청하는 정도를 체크해주세요,3. '건강' 관련 영상에서 주로 시청하는 콘텐츠를 모두 체크해주세요,...,"3. '취미, 레저' 관련 영상에서 주로 시청하는 콘텐츠를 모두 체크해주세요","4. '취미, 레저' 관련 영상에서 자주 시청하는 유튜버가 있으면 적어주세요",유튜브에서 '홈 및 가든' 관련된 영상을 시청하시나요?,1. '홈 및 가든' 보는 영상을 모두 체크해주세요,2. '홈 및 가든' 관련 영상을 시청하는 정도를 체크해주세요,3. '홈 및 가든' 관련 영상에서 주로 시청하는 콘텐츠를 모두 체크해주세요,4. '홈 및 가든' 영상에서 자주 시청하는 유튜버가 있으면 적어주세요,1. 화장품 브랜드 '클리오'를 알고계신가요?,2. '클리오' 제품을 구매해보신 적이 있나요?,3. '클리오' 제품을 앞으로도 계속 이용할 의향이 있으신가요?
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, 독서, 만화 및 애니메...",네,"건강 상태, 여성 건강, 정신 건강, 영양",3.0,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,독서,아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"독서, 영화, 음악, 여행",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, TV, 독서, 만화 및...",아니오,NaN,NaN,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,NaN,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","애완동물, 예술 및 공연, 패션, TV, 영화, 음악, 미용, 건강 및 피트니스, ...",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네


In [4]:
data1.columns = ['나이','전공','색조화장품구매의도','색조화장품다양하게고려','유투브시청목적','관심분야',
                 '건강','건강-소','건강-정도','건강-콘텐츠','건강-유투버',
                 '뉴스','뉴스-소','뉴스-정도','뉴스-콘텐츠','뉴스-유투버',
                 '도서문학','도서문학-소','도서문학-정도','도서문학-콘텐츠','도서문학-유투버',
                 '미용피트니스','미용피트니스-소','미용피트니스-정도','미용피트니스-콘텐츠','미용피트니스-유투버',
                 '쇼핑','쇼핑-소','쇼핑-정도','쇼핑-콘텐츠','쇼핑-유투버',
                 '스포츠','스포츠-소','스포츠-정도','스포츠-콘텐츠','스포츠-유투버',
                 '식음료','식음료-소','식음료-정도','식음료-콘텐츠','식음료-유투버',
                 '애완동물','애완동물-소','애완동물-정도','애완동물-콘텐츠','애완동물-유투버',
                 '여행','여행-소','여행-정도','여행-콘텐츠','여행-유투버',
                 '예술엔터테인먼트','예술엔터테인먼트-소','예술엔터테인먼트-정도','예술엔터테인먼트-콘텐츠','예술엔터테인먼트-유투버',
                 '집단사회','집단사회-소','집단사회-정도','집단사회-콘텐츠','집단사회-유투버',
                 '취미레저','취미레저-소','취미레저-정도','취미레저-콘텐츠','취미레저-유투버',
                 '홈및가든','홈및가든-소','홈및가든-정도','홈및가든-콘텐츠','홈및가든-유투버',
                 '클리오인지여부','클리오구매여부','클리오제품사용지속여부']
data1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, 독서, 만화 및 애니메...",네,"건강 상태, 여성 건강, 정신 건강, 영양",3.0,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,독서,아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"독서, 영화, 음악, 여행",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, TV, 독서, 만화 및...",아니오,NaN,NaN,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,NaN,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","애완동물, 예술 및 공연, 패션, TV, 영화, 음악, 미용, 건강 및 피트니스, ...",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네


In [5]:
data1.나이.value_counts()

23    186
22    185
21    144
20    137
24     77
25     50
19     42
26     22
27     21
28     19
Name: 나이, dtype: int64

In [8]:
data2 = data1.copy()

In [9]:
## 해당 카테고리를 보면 카테고리 이름으로 바꾸기
for i in range(0,len(data2)):
    for j in range(6,71,5):
        if data2.iloc[i, j] == "네":
            data2.iloc[i,j] = data2.columns[j] + '(대)'
        else:
            continue
            
data2.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, 독서, 만화 및 애니메...",건강(대),"건강 상태, 여성 건강, 정신 건강, 영양",3.0,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,독서,아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"독서, 영화, 음악, 여행",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, TV, 독서, 만화 및...",아니오,NaN,NaN,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,NaN,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","애완동물, 예술 및 공연, 패션, TV, 영화, 음악, 미용, 건강 및 피트니스, ...",아니오,NaN,NaN,NaN,...,NaN,NaN,아니오,NaN,NaN,NaN,NaN,네,네,네


In [10]:
## 정도를 int로 바꾸기
# 8 13 18 23 28 33 38 43 48 53 58 63 68
for j in range(8, 73, 5):
    data2.iloc[:,j] = data2.iloc[:,j].fillna(0).astype(int)
data2.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, 독서, 만화 및 애니메...",건강(대),"건강 상태, 여성 건강, 정신 건강, 영양",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,독서,아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"독서, 영화, 음악, 여행",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","엔터테인먼트 뉴스, 가족, 애완동물, 예술 및 공연, 패션, TV, 독서, 만화 및...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","애완동물, 예술 및 공연, 패션, TV, 영화, 음악, 미용, 건강 및 피트니스, ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [11]:
## 관심분야에 텍스트 추가
temp = []
for i in range(0,len(data2)):
    a = data2.iloc[i,5]
    length = len(data2.iloc[i,5].split(','))
    for j in range(0, length):
        temp1 = a.split(',')[j] + '(관심분야)'
        temp.append(temp1)
    data2.관심분야[i] = temp
    temp = []

In [12]:
data2["관심분야"] = data2["관심분야"].apply(lambda x: str(x).replace(" ","").replace("[","").replace("]","").replace("'","") + ',')
data2["관심분야"] = data2["관심분야"].apply(lambda x: x[:-1])
data2["관심분야"] = data2['관심분야'].apply(lambda x: x.split(','))
data2.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태, 여성 건강, 정신 건강, 영양",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


### 소 카테고리 전처리

In [13]:
data22 = data2.copy()

In [14]:
#미용,피트니스: 뷰티 서비스(피부 관리, 경락 등) -> 뷰티서비스
#미용,피트니스: 다이어트(체중감량을 위해)
#미용,피트니스: 피트니스(건강을 위해)
#쇼핑: 잡화(악세사리, 가방, 신발 등)
#스포츠: 개별 스포츠(골프, 경마, 사격, 바둑 등 개인이 참가하는 스포츠)
#스포츠: 경기 득점 및 통계(예. 손흥민 득점 장면)
#스포츠: 팀 스포츠(야구, 축구, 농구 등 팀이 참가하는 스포츠)
#식음료: 음식(끼니용)
#식음료: 디저트(과자, 아이스크림, 마카롱 등 간식)
#여행: 여행 가방, 여행 용품
#여행: 여행사, 여행 서비스 (투어여행, 가이드 동반 여행 및 휴가, 여행자 숙소, 휴양상품)
#여행: 특수관광(관광농업, 도전여행, 생태관광, 포도밭 및 와인관람)
#예술, 엔터테인먼트: 공연 예술(춤, 마술, 뮤지컬 관람, 서커스, 연극, 오페라)
#예술, 엔터테인먼트: 드라마 영화(가족, 로맨스, 코미디)
#예술, 엔터테인먼트: 애니메이션 영화(예. 디즈니 영화, 지브리 영화)
#예술, 엔터테인먼트: TV 가족용 프로그램(전체 연령가 프로그램)
#예술, 엔터테인먼트: TV 리얼리티쇼(예. 나혼자산다, 전참시 등)
#예술, 엔터테인먼트: TV 버라이어티쇼(예. 무한도전 등)
#예술, 엔터테인먼트: 만화 영화(예. 짱구 극장판, 명탐정 코난 극장판 등)
#예술, 엔터테인먼트: 애니메이션 및 만화 (예. 다다다, 학원앨리스 등)
#집단, 사회: 가족관계 및 이성교제 (가족, 결혼, 로맨스, 우정, 인간관계 문제)
#집단, 사회: 사회과학(경제학, 심리학, 정치학)
#취미, 레저: 드라마 및 영화 추천(예. 넷플릭스 추천, 미드 추천 등)
data22.iloc[:,22] = data22.iloc[:,22].apply(lambda x: x.replace('뷰티 서비스(피부 관리, 경락 등)', '뷰티 서비스') if pd.notnull(x) else x)
data22.iloc[:,22] = data22.iloc[:,22].apply(lambda x: x.replace('다이어트(체중감량을 위해)', '다이어트') if pd.notnull(x) else x)
data22.iloc[:,22] = data22.iloc[:,22].apply(lambda x: x.replace('피트니스(건강을 위해)', '피트니스') if pd.notnull(x) else x)
data22.iloc[:,27] = data22.iloc[:,27].apply(lambda x: x.replace('잡화(악세사리, 가방, 신발 등)', '잡화') if pd.notnull(x) else x)
data22.iloc[:,32] = data22.iloc[:,32].apply(lambda x: x.replace('개별 스포츠(골프, 경마, 사격, 바둑 등 개인이 참가하는 스포츠)', '개별 스포츠') if pd.notnull(x) else x)
data22.iloc[:,32] = data22.iloc[:,32].apply(lambda x: x.replace('경기 득점 및 통계(예. 손흥민 득점 장면)', '경기 득점 및 통계') if pd.notnull(x) else x)
data22.iloc[:,32] = data22.iloc[:,32].apply(lambda x: x.replace('팀 스포츠(야구, 축구, 농구 등 팀이 참가하는 스포츠)', '팀 스포츠') if pd.notnull(x) else x)
data22.iloc[:,37] = data22.iloc[:,37].apply(lambda x: x.replace('음식(끼니용)', '음식') if pd.notnull(x) else x)
data22.iloc[:,37] = data22.iloc[:,37].apply(lambda x: x.replace('디저트(과자, 아이스크림, 마카롱 등 간식)', '디저트') if pd.notnull(x) else x)
data22.iloc[:,47] = data22.iloc[:,47].apply(lambda x: x.replace('여행 가방, 여행 용품', '여행 가방 및 여행 용품') if pd.notnull(x) else x)
data22.iloc[:,47] = data22.iloc[:,47].apply(lambda x: x.replace('여행사, 여행 서비스 (투어여행, 가이드 동반 여행 및 휴가, 여행자 숙소, 휴양상품)', '여행사 및 여행 서비스') if pd.notnull(x) else x)
data22.iloc[:,47] = data22.iloc[:,47].apply(lambda x: x.replace('특수관광(관광농업, 도전여행, 생태관광, 포도밭 및 와인관람)', '특수관광') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('공연 예술(춤, 마술, 뮤지컬 관람, 서커스, 연극, 오페라)', '공연 예술') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('드라마 영화(가족, 로맨스, 코미디)', '드라마 영화') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('애니메이션 영화(예. 디즈니 영화, 지브리 영화)', '애니메이션 영화') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('TV 가족용 프로그램(전체 연령가 프로그램)', 'TV 가족용 프로그램') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('TV 리얼리티쇼(예. 나혼자산다, 전참시 등)', 'TV 리얼리티쇼') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('TV 버라이어티쇼(예. 무한도전 등)', 'TV 버라이어티쇼') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('만화 영화(예. 짱구 극장판, 명탐정 코난 극장판 등)', '만화 영화') if pd.notnull(x) else x)
data22.iloc[:,52] = data22.iloc[:,52].apply(lambda x: x.replace('애니메이션 및 만화 (예. 다다다, 학원앨리스 등)', '애니메이션 및 만화') if pd.notnull(x) else x)
data22.iloc[:,57] = data22.iloc[:,57].apply(lambda x: x.replace('가족관계 및 이성교제 (가족, 결혼, 로맨스, 우정, 인간관계 문제)', '가족관계 및 이성교제') if pd.notnull(x) else x)
data22.iloc[:,57] = data22.iloc[:,57].apply(lambda x: x.replace('사회과학(경제학, 심리학, 정치학)', '사회과학') if pd.notnull(x) else x)
data22.iloc[:,62] = data22.iloc[:,62].apply(lambda x: x.replace('드라마 및 영화 추천(예. 넷플릭스 추천, 미드 추천 등)', '드라마 및 영화 추천') if pd.notnull(x) else x)
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태, 여성 건강, 정신 건강, 영양",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [15]:
## 소카테고리 맨 뒤에 콤마 추가하기
for i in range(0, len(data22)):
    for j in range(7, 72, 5):
        if data22.iloc[i,j-1] != '아니오':
            data22.iloc[i,j] = data22.iloc[i,j] + ','
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태, 여성 건강, 정신 건강, 영양,",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [16]:
## 소 추가
for i in range(7,72,5):
    test = data22.iloc[:,i].str.replace(',', '(소),')
    data22.iloc[:,i] = test.apply(lambda x: x[:-1] if pd.notnull(x) else x)
    continue
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태(소), 여성 건강(소), 정신 건강(소), 영양(소)",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


### 콘텐츠 전처리

In [17]:
## 대 카테고리를 본다고 할 때, 맨 뒤에 콤마 추가하기
for i in range(0, len(data22)):
    for j in range(9, 74, 5):
        if data22.iloc[i,j-3] != '아니오':
            data22.iloc[i,j] = data22.iloc[i,j] + ','
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태(소), 여성 건강(소), 정신 건강(소), 영양(소)",3,"리뷰 및 추천, 하울, 언박싱, 정보 전달, ASMR, 음악, 여행, 웹드라마, 유...",...,"VLOG, 리뷰 및 추천, 정보 전달, 유머, 여행, 춤, 음악, 영화, 하울, 웹...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG, 리뷰 및 추천, 정보 전달, 유머,","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [71]:
## 카테고리+컨텐츠 추가
for i in range(9, 74, 5):
    name=data22.columns[i-3]
    column_name = data22.columns[i]
    test = data22[column_name].apply(lambda x: x.replace(',', '(' + name + '콘텐츠),') if pd.notnull(x) else x)
    data22.iloc[:,i] = test.apply(lambda x: x[:-1] if pd.notnull(x) else x)
    continue
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태(소), 여성 건강(소), 정신 건강(소), 영양(소)",3,"리뷰 및 추천(건강콘텐츠), 하울(건강콘텐츠), 언박싱(건강콘텐츠), 정보 전달(건...",...,"VLOG(취미레저콘텐츠), 리뷰 및 추천(취미레저콘텐츠), 정보 전달(취미레저콘텐츠...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG(취미레저콘텐츠), 리뷰 및 추천(취미레저콘텐츠), 정보 전달(취미레저콘텐츠...","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [18]:
## 컨텐츠만 추가
for i in range(9, 74, 5):
    name=data22.columns[i-3]
    column_name = data22.columns[i]
    test = data22[column_name].apply(lambda x: x.replace(',',  '(콘텐츠),') if pd.notnull(x) else x)
    data22.iloc[:,i] = test.apply(lambda x: x[:-1] if pd.notnull(x) else x)
    continue
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태(소), 여성 건강(소), 정신 건강(소), 영양(소)",3,"리뷰 및 추천(콘텐츠), 하울(콘텐츠), 언박싱(콘텐츠), 정보 전달(콘텐츠), A...",...,"VLOG(콘텐츠), 리뷰 및 추천(콘텐츠), 정보 전달(콘텐츠), 유머(콘텐츠), ...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG(콘텐츠), 리뷰 및 추천(콘텐츠), 정보 전달(콘텐츠), 유머(콘텐츠)","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


In [20]:
data22.to_pickle('설문지_안곱함.pkl')

In [22]:
data22 = pd.read_pickle('설문지_안곱함.pkl')
data22.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,유투브시청목적,관심분야,건강,건강-소,건강-정도,건강-콘텐츠,...,취미레저-콘텐츠,취미레저-유투버,홈및가든,홈및가든-소,홈및가든-정도,홈및가든-콘텐츠,홈및가든-유투버,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,정보 탐색을 위해서,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",건강(대),"건강 상태(소), 여성 건강(소), 정신 건강(소), 영양(소)",3,"리뷰 및 추천(콘텐츠), 하울(콘텐츠), 언박싱(콘텐츠), 정보 전달(콘텐츠), A...",...,"VLOG(콘텐츠), 리뷰 및 추천(콘텐츠), 정보 전달(콘텐츠), 유머(콘텐츠), ...",NaN,아니오,NaN,0,NaN,NaN,네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,페미니즘,[독서(관심분야)],아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,아니오,아니오,아니오
2,19,치의예과,네,네,정보 탐색을 위해서,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네
3,19,수학교육전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...",아니오,NaN,0,NaN,...,"VLOG(콘텐츠), 리뷰 및 추천(콘텐츠), 정보 전달(콘텐츠), 유머(콘텐츠)","사나고, 이연",아니오,NaN,0,NaN,NaN,네,네,네
4,19,빅데이터경영통계전공,네,네,"시간 때우기용, 정보 탐색을 위해서, 힐링","[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...",아니오,NaN,0,NaN,...,NaN,NaN,아니오,NaN,0,NaN,NaN,네,네,네


### 콘텐츠: 정도에 따라 곱하기

In [23]:
data33=data22.copy()

In [24]:
data33.iloc[:,7]=data33.iloc[:,7].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,12]=data33.iloc[:,12].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,17]=data33.iloc[:,17].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,22]=data33.iloc[:,22].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,27]=data33.iloc[:,27].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,32]=data33.iloc[:,32].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,37]=data33.iloc[:,37].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,42]=data33.iloc[:,42].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,47]=data33.iloc[:,47].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,52]=data33.iloc[:,52].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,57]=data33.iloc[:,57].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,62]=data33.iloc[:,62].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,67]=data33.iloc[:,67].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,9]=data33.iloc[:,9].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,14]=data33.iloc[:,14].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,19]=data33.iloc[:,19].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,24]=data33.iloc[:,24].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,29]=data33.iloc[:,29].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,34]=data33.iloc[:,34].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,39]=data33.iloc[:,39].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,44]=data33.iloc[:,44].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,49]=data33.iloc[:,49].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,54]=data33.iloc[:,54].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,59]=data33.iloc[:,59].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,64]=data33.iloc[:,64].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)
data33.iloc[:,69]=data33.iloc[:,69].apply(lambda x: x.replace(" ","") if pd.notnull(x) else x)

In [198]:
## 대+소+컨텐츠
contents = []
c=[]

for i in range(0, len(data33)):
    for j in range(6, 71, 5):
        if data33.iloc[i,j] == '아니오':
            continue
        else:
            a = data33.iloc[i,j] + ","+ data33.iloc[i,j+1] + "," + data33.iloc[i,j+3] 
            b = a.split(",") * data33.iloc[i,j+2]
            c.extend(b)
    contents.append(c)
    c=[]
    continue
        

In [139]:
## 소+컨텐츠(곱하기)
contents = []
c=[]

for i in range(0, len(data33)):
    for j in range(6, 71, 5):
        if data33.iloc[i,j] == '아니오':
            continue
        else:
            a = data33.iloc[i,j+1] + "," + data33.iloc[i,j+3] 
            b = a.split(",") * data33.iloc[i,j+2]
            c.extend(b)
    contents.append(c)
    c=[]
    continue

In [25]:
## 소+컨텐츠(안곱함)
contents = []
c=[]

for i in range(0, len(data33)):
    for j in range(6, 71, 5):
        if data33.iloc[i,j] == '아니오':
            continue
        else:
            a = data33.iloc[i,j+1] + "," + data33.iloc[i,j+3] 
            b = a.split(",") 
            c.extend(b)
    contents.append(c)
    c=[]
    continue

In [27]:
final = pd.DataFrame(columns=['나이','전공','색조화장품구매의도','색조화장품다양하게고려','관심분야','콘텐츠'])
final["나이"] = data33["나이"]
final["전공"] = data33["전공"]
final["색조화장품구매의도"] = data33["색조화장품구매의도"]
final["색조화장품다양하게고려"] = data33['색조화장품다양하게고려']
final['관심분야'] = data33["관심분야"]
final["콘텐츠"] = contents
final["클리오인지여부"] = data33["클리오인지여부"]
final["클리오구매여부"] = data33["클리오구매여부"]
final["클리오제품사용지속여부"] = data33["클리오제품사용지속여부"]
final.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,관심분야,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,[독서(관심분야)],"[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [28]:
final.to_pickle("설문지_최종_대빼고_곱안함.pkl")
data33.to_pickle("설문지_최종_모든열_대빼고_곱안함.pkl")

In [29]:
final1 = pd.read_pickle("설문지_최종_대빼고_곱안함.pkl")
every1 = pd.read_pickle("설문지_최종_모든열_대빼고_곱안함.pkl")

In [30]:
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,관심분야,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,[독서(관심분야)],"[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야)]","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [31]:
## 관심분야+콘텐츠 합치기
for i in range(0, len(final1)):
    final1.관심분야[i].extend(final1.콘텐츠[i])

final1.rename(columns= {"관심분야" : "최종corpus"}, inplace=True )
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,최종corpus,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 가십뉴스(...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [32]:
final1.to_pickle("설문지_최종corpus_대빼고_곱안함.pkl")
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,최종corpus,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 가십뉴스(...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [105]:
final1 = pd.read_pickle("설문지_최종corpus_대빼고_곱안함.pkl")
str(final1.최종corpus[1])

"['독서(관심분야)', '페미니즘(소)', '리뷰및추천(콘텐츠)', '정보전달(콘텐츠)']"

### 관심분야 개수 늘리기

In [5]:
# 카테고리(정도)
ind = [8,13,18,23,28,33,38,43,48,53,58,63,68]
Degree = every1.iloc[:,ind]
Degree.head()

,건강-정도,뉴스-정도,도서문학-정도,미용피트니스-정도,쇼핑-정도,스포츠-정도,식음료-정도,애완동물-정도,여행-정도,예술엔터테인먼트-정도,집단사회-정도,취미레저-정도,홈및가든-정도
0,3,5,4,4,4,0,0,5,4,0,5,5,0
1,0,0,5,0,0,0,0,0,0,0,0,0,0
2,0,3,4,0,0,0,0,0,2,5,0,0,0
3,0,2,0,2,2,1,3,3,1,5,0,3,0
4,0,0,0,3,3,0,4,3,3,5,0,0,0


In [7]:
dg = Degree.sum(axis = 1)

for i in range(0, len(final1)) :
    count = round(dg[i] / len(final1['관심분야'][i]))
    count = int(count)
    final1["관심분야"][i].extend(final1['관심분야'][i] * count)
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,관심분야,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심...","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠), 페미니즘(소), 리뷰및추...",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 독서(관심...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [8]:
for i in range(0, len(final1)):
    final1.관심분야[i].extend(final1.콘텐츠[i])

In [9]:
final1.rename(columns= {"관심분야" : "최종corpus"}, inplace=True )
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,최종corpus,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심...","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠), 페미니즘(소), 리뷰및추...",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 독서(관심...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [10]:
final1.to_pickle("설문지_최종corpus_대빼고_관심분야곱.pkl")

In [11]:
final2 = pd.read_pickle("설문지_최종corpus_대빼고_관심분야곱.pkl")
final2.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,최종corpus,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심분야), 독서(관심...","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠), 페미니즘(소), 리뷰및추...",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 독서(관심...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


### corpus 수 계산

In [65]:
final11 = final1.copy()

In [66]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
a = mlb.fit_transform(final11.최종corpus)

In [67]:
from collections import Counter

In [68]:
final1.head()

,나이,전공,색조화장품구매의도,색조화장품다양하게고려,최종corpus,콘텐츠,클리오인지여부,클리오구매여부,클리오제품사용지속여부
0,19,글로벌리더학과,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[건강상태(소), 여성건강(소), 정신건강(소), 영양(소), 리뷰및추천(콘텐츠),...",네,네,네
1,19,이런것좀 하지마시죠 ^^,아니오,아니오,"[독서(관심분야), 페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]","[페미니즘(소), 리뷰및추천(콘텐츠), 정보전달(콘텐츠)]",아니오,아니오,아니오
2,19,치의예과,네,네,"[독서(관심분야), 영화(관심분야), 음악(관심분야), 여행(관심분야), 가십뉴스(...","[가십뉴스(소), 정치뉴스(소), 음악(콘텐츠), 여행(콘텐츠), 메이크업튜토리얼(...",네,네,네
3,19,수학교육전공,네,네,"[엔터테인먼트뉴스(관심분야), 가족(관심분야), 애완동물(관심분야), 예술및공연(관...","[가십뉴스(소), 건강정보(소), 세계뉴스(소), 정치뉴스(소), 정보전달(콘텐츠)...",네,네,네
4,19,빅데이터경영통계전공,네,네,"[애완동물(관심분야), 예술및공연(관심분야), 패션(관심분야), TV(관심분야), ...","[뷰티(소), 패션(소), 뷰티서비스(소), 다이어트(소), 헤어(소), 리뷰및추천...",네,네,네


In [69]:
df = pd.DataFrame(columns=mlb.classes_)
for i in range(0, len(final1)):
    result = Counter(final1.최종corpus[i])
    df = df.append(result, ignore_index=True).fillna(0)

df.head()

,24시간라이브방송(소),3D펜으로물건제작하기등(소),ASMR(콘텐츠),DIY(소),E-sports(소),Earwax(소),E스포츠(게임)봅니다.(소),GRWM(겟레디위드미)(콘텐츠),OST(소),QnA(소),...,헤르페스치료방법등등그리고약사들이운영하는채널도시청(소),헤어(소),헬스(소),홈요가영상(소),홈트(소),홈트(콘텐츠),홈트레이닝(운동)(소),홈트레이닝따라핮니다(소),홈피트니스(소),화장품(소)
0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0


In [70]:
df['나이']=final1['나이']
df['색조화장품구매의도'] = final1['색조화장품구매의도'].map({'네': 1, '아니오': 0})
df['색조화장품다양하게고려'] = final1['색조화장품다양하게고려'].map({'네': 1, '아니오': 0})
df['클리오인지여부'] =final1['클리오인지여부'].map({'네': 1, '아니오': 0})
df['클리오구매여부'] = final1['클리오구매여부'].map({'네': 1, '아니오': 0})
df.head()

,24시간라이브방송(소),3D펜으로물건제작하기등(소),ASMR(콘텐츠),DIY(소),E-sports(소),Earwax(소),E스포츠(게임)봅니다.(소),GRWM(겟레디위드미)(콘텐츠),OST(소),QnA(소),...,홈트(콘텐츠),홈트레이닝(운동)(소),홈트레이닝따라핮니다(소),홈피트니스(소),화장품(소),나이,색조화장품구매의도,색조화장품다양하게고려,클리오인지여부,클리오구매여부
0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,19,1,1,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,1,1,1,1
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,19,1,1,1,1
4,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,19,1,1,1,1


In [71]:
df1 = df.loc[:,df.sum(axis=0)!=1].loc[:,df.sum(axis=0)!=2].loc[:,df.sum(axis=0)!=3].loc[:,df.sum(axis=0)!=4].loc[:,df.sum(axis=0)!=5]
df1.head()

,ASMR(콘텐츠),GRWM(겟레디위드미)(콘텐츠),OST(소),TV(관심분야),TV가족용프로그램(소),TV다큐멘터리(소),TV드라마(소),TV리얼리티쇼(소),TV버라이어티쇼(소),TV코미디(소),...,피트니스(소),하울(콘텐츠),헤어(소),홈트(콘텐츠),화장품(소),나이,색조화장품구매의도,색조화장품다양하게고려,클리오인지여부,클리오구매여부
0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,1.0,0.0,1.0,19,1,1,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,0,0,0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,1,1,1,1
3,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,19,1,1,1,1
4,3.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,2.0,1.0,2.0,1.0,19,1,1,1,1


In [72]:
df1.to_csv("설문지_corpus개수.csv", encoding='utf8', index=False)

In [74]:
df1 = pd.read_csv("설문지_corpus개수.csv",engine='python', encoding='utf8')
df1.head()

,ASMR(콘텐츠),GRWM(겟레디위드미)(콘텐츠),OST(소),TV(관심분야),TV가족용프로그램(소),TV다큐멘터리(소),TV드라마(소),TV리얼리티쇼(소),TV버라이어티쇼(소),TV코미디(소),...,피트니스(소),하울(콘텐츠),헤어(소),홈트(콘텐츠),화장품(소),나이,색조화장품구매의도,색조화장품다양하게고려,클리오인지여부,클리오구매여부
0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,1.0,0.0,1.0,19,1,1,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,0,0,0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19,1,1,1,1
3,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,19,1,1,1,1
4,3.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,2.0,1.0,2.0,1.0,19,1,1,1,1


In [75]:
df1['타겟'] = df1['색조화장품구매의도'] + df1['색조화장품다양하게고려']
df1['타겟'] = df1['타겟'].map({0: 0, 1: 0, 2:1})
df1.head()

,ASMR(콘텐츠),GRWM(겟레디위드미)(콘텐츠),OST(소),TV(관심분야),TV가족용프로그램(소),TV다큐멘터리(소),TV드라마(소),TV리얼리티쇼(소),TV버라이어티쇼(소),TV코미디(소),...,하울(콘텐츠),헤어(소),홈트(콘텐츠),화장품(소),나이,색조화장품구매의도,색조화장품다양하게고려,클리오인지여부,클리오구매여부,타겟
0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,0.0,1.0,19,1,1,1,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,0,0,0,0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19,1,1,1,1,1
3,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,19,1,1,1,1,1
4,3.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,19,1,1,1,1,1


In [81]:
df1.groupby(df1['나이']).sum()['뮤직비디오(소)']

나이
19    22.0
20    80.0
21    67.0
22    85.0
23    79.0
24    30.0
25    18.0
26     8.0
27     4.0
28     1.0
Name: 뮤직비디오(소), dtype: float64

In [87]:
df1.groupby(df1['나이']).sum()['TV드라마(소)']

나이
19    10.0
20    36.0
21    38.0
22    48.0
23    34.0
24    15.0
25     9.0
26     1.0
27     4.0
28     2.0
Name: TV드라마(소), dtype: float64

# word2vec

### corpus

In [44]:
item_corpus = []

for i in final1.최종corpus :
    
    for j in range(0, 5) :
    
        item = random.sample(i, len(i))
        item_corpus.append(item)
    
print(len(item_corpus))

4415


### word2vec

In [53]:
# start time
start_time = time.time() 

model = Word2Vec(item_corpus,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=300,     # 벡터 크기
                 window=1,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=15,  # 사용할 단어의 최소 빈도(1회 이하 단어 무시)
                 workers=2)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)
# time check
print("\n ---------- %s seconds ----------" %(round(time.time() - start_time)))


 ---------- 1134 seconds ----------


In [54]:
## 전체 학습에 사용된 단어의 개수
vocab = list(model.wv.vocab)
len(vocab)

142

In [55]:
## 데이터프레임 저장
vocab = list(model.wv.vocab)
X = model[vocab]

Clustering = pd.DataFrame(X, index = vocab)
Clustering.to_csv('설문지_word2vec_300_1_2_daex_곱x_5.csv')